# Trading Bot

In [2]:
!pip install robin_stocks

You should consider upgrading via the 'c:\users\aron gosch\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [8]:
import robin_stocks.robinhood as r
import pandas as pd
import time

## 1) Login/access Robin hood stocks

In [ ]:
username = 'user@mail.com'
password = 'password'

login = r.login(username,password)

In [9]:
r.build_holdings()

Exception: build_holdings can only be called when logged in

In [ ]:
#Utilities
def visualize_price(ticker_list, span = 'year', bounds = 'regular'):   
    for t in range(len(ticker_list)):
        name = str(r.get_name_by_symbol(ticker_list[t]))
        hist = r.stocks.get_historicals(ticker_list[t], span=span, bounds=bounds)
        hist_df = pd.DataFrame()
        for i in range(len(hist)):
            df = pd.DataFrame(hist[i], index = [i])
            hist_df = pd.concat([hist_df,df])
        hist_df.begins_at = pd.to_datetime(hist_df.begins_at, infer_datetime_format=True)
        hist_df.open_price = hist_df.open_price.astype('float32')
        hist_df.close_price = hist_df.close_price.astype('float32')
        hist_df.high_price = hist_df.high_price.astype('float32')
        hist_df.low_price = hist_df.low_price.astype('float32')

        ax = hist_df.plot(x = 'begins_at', y = 'open_price', figsize = (16,8))
        ax.fill_between(hist_df.begins_at, hist_df.low_price, hist_df.high_price, alpha=0.5)
        ax.set_xlabel('Date')
        ax.set_ylabel('Price (USD)')
        ax.legend([ticker_list[t]])
        ax.set_title(name)
    return

def extract_list():
    ticker_list = list(r.build_holdings().keys())
    return ticker_list

In [4]:
def trading_bot(trading_dict):
    """If the stock price has a drop over a certain percentage the bot will execute a buy. 
    Conversely, if the stock price has a rise over a certain percentage the bot will execute a sell."""
    holdings = r.build_holdings()
    holdings_df = pd.DataFrame()
    for i in range(len(holdings)):
        ticker = list(holdings.items())[i][0]
        holding_df = pd.DataFrame(list(holdings.items())[i][1], index = [i])
        holding_df['ticker'] = ticker
        holdings_df = pd.concat([holdings_df, holding_df])
    holdings_df = holdings_df[['ticker', 'price', 'quantity', 'percent_change','average_buy_price', 'equity', 'equity_change','pe_ratio', 'type', 'name', 'id' ]]

    for j in range(len(trading_dict)):
        holding_df = holdings_df[holdings_df.ticker == list(trading_dict.keys())[j]]
        if holding_df['percent_change'].astype('float32')[0] <= list(trading_dict.values())[j][0]:
            buy_string = 'Buying ' + str(holding_df['ticker'][0]) + ' at ' + time.ctime()
            print(buy_string)
            r.orders.order_buy_market(holding_df['ticker'][0],1,timeInForce= 'gfd')
        else:
            print('Nothing to buy')

        if holding_df['percent_change'].astype('float32')[0] >= list(trading_dict.values())[j][1]:
            sell_string = 'Buying ' + str(holding_df['ticker'][0]) + ' at ' + time.ctime()
            print(sell_string)
            r.orders.order_sell_market(holding_df['ticker'][0],1,timeInForce= 'gfd')
        else:
            print('Nothing to sell')

In [5]:
trading_dict = {'KMI': [-0.50, 0.50]}
holdings_df = trading_bot(trading_dict)

AttributeError: module 'robin_stocks' has no attribute 'build_holdings'

Buy 10 shares of Apple at market price

robin_stocks.order_buy_market('AAPL',10)

Sell half a Bitcoin is price reaches 10,000

robin_stocks.order_sell_crypto_limit('BTC',0.5,10000)

Buy $500 worth of Bitcoin

robin_stocks.order_buy_crypto_by_price('BTC',500)

Buy 5 150 dollar May 1st, 2020 SPY puts if the price per contract is 1.00 dollar. Good until cancelled.

robin_stocks.order_buy_option_limit('open','debit',1.00,'SPY',5,'2020-05-01',150,'put','gtc')